In [5]:
# adapted from https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, RepeatVector, Dense
import numpy as np
import wandb
from wandb.keras import WandbCallback

wandb.init()
config = wandb.config

class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

# Parameters for the model and dataset.
config.training_size = 50000
config.digits = 30
config.hidden_size = 128
config.batch_size = 128

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
maxlen = config.digits + config.digits

# All the numbers, plus sign and space for padding.
chars = '0123456789+$-,/.ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuwxyv '
ctable = CharacterTable(chars)


import pandas as pd
df = pd.read_csv("sale_prices.csv")
df = df.fillna(0)



raw_price = df["Price on site"]
parced_price = []

int_min_price = [int(price) for price in df["Min price"]]
int_max_price = [int(price) for price in df["Max price"]]
for idx in range(len(int_min_price)):
    toappend = str(int_min_price[idx])
    toappend+=' '*(30 - len(str(int_min_price[idx])) - len(str(int_max_price[idx])))
    toappend+=str(int_max_price[idx])
    parced_price.append(toappend)

print('Total prices:', len(raw_price))

print('Vectorization...')
x = np.zeros((len(raw_price), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(raw_price), config.digits + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(raw_price):
    x[i] = ctable.encode(str(sentence), maxlen)
for i, sentence in enumerate(parced_price):
    y[i] = ctable.encode(str(sentence), config.digits + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

model = Sequential()
model.add(LSTM(config.hidden_size, input_shape=(maxlen, len(chars))))
model.add(RepeatVector(config.digits + 1))
model.add(LSTM(config.hidden_size, return_sequences=True))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
             metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=config.batch_size,
              epochs=1,
              validation_data=(x_val, y_val),callbacks=[WandbCallback()])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print(guess)


Total prices: 638
Vectorization...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               100864    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 31, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 31, 128)           131584    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 31, 68)            8772      
Total params: 241,220
Trainable params: 241,220
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 1
Train on 575 samples, validate on 63 samples
Epoch 1/1
575/575 [==============================] - 1s 1ms/step - loss: 4.0502 - accuracy: 0.4752 - v

Q $14,900                                                      T 14900                    14900  ☒                                
Q $987,909                                                     T 987909                  987909  ☒                                
Q $22,500,000                                                  T 22500000              22500000  ☒                                

--------------------------------------------------
Iteration 6
Train on 575 samples, validate on 63 samples
Epoch 1/1
575/575 [==============================] - 0s 395us/step - loss: 1.3002 - accuracy: 0.6141 - val_loss: 1.2531 - val_accuracy: 0.6262
Q $280,000                                                     T 280000                  280000  ☒                                
Q $3,585,000+                                                  T 3585000                      0  ☒                                
Q $299,000                                                     T 299000                  2990

575/575 [==============================] - 0s 363us/step - loss: 1.1918 - accuracy: 0.6439 - val_loss: 1.1324 - val_accuracy: 0.6887
Q $279,900                                                     T 279900                  279900  ☒ 90000                          
Q $1,350,000                                                   T 1350000                1350000  ☒ 90000                          
Q $299,000                                                     T 299000                  299000  ☒ 90000                          
Q $14,900                                                      T 14900                    14900  ☒ 90000                          
Q From $414,300                                                T 414300                       0  ☒ 90000                          
Q $5,950,000                                                   T 5950000                5950000  ☒ 90000                          
Q $4,875,000                                                   T 4875000         

Q $22,000                                                      T 22000                    22000  ☒ 190000                         
Q $299,000                                                     T 299000                  299000  ☒ 190000                         
Q $925,000                                                     T 925000                  925000  ☒ 190000                         
Q $70,000                                                      T 70000                    70000  ☒ 190000                         
Q $22,000                                                      T 22000                    22000  ☒ 190000                         

--------------------------------------------------
Iteration 17
Train on 575 samples, validate on 63 samples
Epoch 1/1
575/575 [==============================] - 0s 375us/step - loss: 1.1336 - accuracy: 0.6954 - val_loss: 1.0810 - val_accuracy: 0.7097
Q $987,909                                                     T 987909                  987

Q $22,000                                                      T 22000                    22000  ☒ 190000                         

--------------------------------------------------
Iteration 22
Train on 575 samples, validate on 63 samples
Epoch 1/1
575/575 [==============================] - 0s 405us/step - loss: 1.1152 - accuracy: 0.6954 - val_loss: 1.0648 - val_accuracy: 0.7097
Q $3,585,000+                                                  T 3585000                      0  ☒ 190000                         
Q $4,310,000+                                                  T 4310000                      0  ☒ 190000                         
Q $4,310,000+                                                  T 4310000                      0  ☒ 190000                         
Q -                                                            T 0                            0  ☒ 190000                         
Q -                                                            T 0                          

...

Q $1,850,000                                                   T 1850000                1850000  ☒ 199000                   900000

--------------------------------------------------
Iteration 177
Train on 575 samples, validate on 63 samples
Epoch 1/1
575/575 [==============================] - 0s 366us/step - loss: 0.6821 - accuracy: 0.7469 - val_loss: 0.6438 - val_accuracy: 0.7573
Q $3,585,000+                                                  T 3585000                      0  ☒ 199000                   900000
Q $35,000                                                      T 35000                    35000  ☒ 199000                   900000
Q $1,595,000                                                   T 1595000                1595000  ☒ 199000                   900000
Q From $414,300                                                T 414300                       0  ☒ 199000                   900000
Q $2,275,000                                                   T 2275000                227

Q $75,000                                                      T 75000                    75000  ☒ 199000                   900000
Q $389,000                                                     T 389000                  389000  ☒ 199000                   900000
Q $339,900                                                     T 339900                  339900  ☒ 199000                   900000
Q $500,000                                                     T 500000                  500000  ☒ 199000                   900000
Q $1,850,000                                                   T 1850000                1850000  ☒ 199000                   900000
Q $299,000                                                     T 299000                  299000  ☒ 199000                   900000
Q $232,000                                                     T 232000                  232000  ☒ 199000                   900000
Q $22,500,000                                                  T 22500000          

Q $79,900                                                      T 79900                    79900  ☒ 199000                   900000
Q $299,000                                                     T 299000                  299000  ☒ 199000                   900000
Q $435,000                                                     T 435000                  435000  ☒ 199000                   900000
Q $500,000                                                     T 500000                  500000  ☒ 199000                   900000
Q $179,500                                                     T 179500                  179500  ☒ 199000                   900000

--------------------------------------------------
Iteration 188
Train on 575 samples, validate on 63 samples
Epoch 1/1
575/575 [==============================] - 0s 365us/step - loss: 0.6778 - accuracy: 0.7472 - val_loss: 0.6392 - val_accuracy: 0.7573
Q $179,500                                                     T 179500                  17

575/575 [==============================] - 0s 359us/step - loss: 0.6850 - accuracy: 0.7456 - val_loss: 0.6538 - val_accuracy: 0.7573
Q $3,000,000                                                   T 3000000                3000000  ☒ 199000                   900000
Q $210,000                                                     T 210000                  210000  ☒ 199000                   900000
Q $70,000                                                      T 70000                    70000  ☒ 199000                   900000
Q $1,350,000                                                   T 1350000                1350000  ☒ 199000                   900000
Q $739,900                                                     T 739900                  739900  ☒ 199000                   900000
Q $5,950,000                                                   T 5950000                5950000  ☒ 199000                   900000
Q $2,000,000                                                   T 2000000         

Q $389,000                                                     T 389000                  389000  ☒ 199000                   900000
Q $97,500                                                      T 97500                    97500  ☒ 199000                   900000
Q $1,450,000                                                   T 1450000                1450000  ☒ 199000                   900000
Q $154,580                                                     T 154580                  154580  ☒ 199000                   900000
Q $79,900                                                      T 79900                    79900  ☒ 199000                   900000

--------------------------------------------------
Iteration 199
Train on 575 samples, validate on 63 samples
Epoch 1/1
575/575 [==============================] - 0s 378us/step - loss: 0.6838 - accuracy: 0.7463 - val_loss: 0.6445 - val_accuracy: 0.7532
Q $179,500                                                     T 179500                  17